## 线性回归
线性回归假设目标值与特征之间线性相关，即满足一个多元一次方程。通过构建代价函数来求解参数$\theta$，通常写为
$$\hat{y}=\theta_0+\theta_1x_1+\theta_2x_2+\theta_3x_3+...+\theta_nx_n$$

对特征矩阵添加一列$x_0=1$，即可变成$\hat{y}=\theta_0x_0+\theta_1x_1+\theta_2x_2+\theta_3x_3+...+\theta_nx_n$，也可以写为$\hat{y}=X\theta$。代价函数为$J(\theta)=\frac{1}{m}\sum_{i=1}^m(\hat{y_i}-y_i)^2$
求解方法：
- 最小二乘法：
  代价函数对$\theta$求导，求得代价函数取最小值时，$\theta=(X^TX)^{-1}X^Ty$。
- 梯度下降法：
  同样是代价函数对$\theta$求导和更新$\theta$，使代价函数不断逼近最小值的过程。$\theta_i=\theta_i-\alpha\frac{∂J}{∂\theta}$，其中$\frac{∂J}{∂\theta}=\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})\cdot x_i)$


In [1]:
from TV_examine.algorithm import linearregression
from TV_examine.data_preprocessing import preprocessing, PCA
from TV_examine.model_assessment import reg, divide_data
import matplotlib.pyplot as plt
import numpy as np

## 导入数据集
这里使用波士顿房价数据集进行评估，先大致查看数据集，可知其有13个特征，506个样本。

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
boston.feature_names, boston.data.shape, boston.target.shape

(array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
        'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7'), (506, 13), (506,))

In [4]:
X = boston.data
y = boston.target

In [5]:
preprocessing.data_analysis(X)

均值： 3.614   11.364   11.137    0.069    0.555    6.285   68.575    3.795    9.549   408.237   18.456   356.674   12.653   
中位数： 0.257    0.000    9.690    0.000    0.538    6.208   77.500    3.207    5.000   330.000   19.050   391.440   11.360   
标准差： 8.602   23.322    6.860    0.254    0.116    0.703   28.149    2.106    8.707   168.537    2.165   91.295    7.141   
极差：88.970   100.000   27.280    1.000    0.486    5.219   97.100   10.997   23.000   524.000    9.400   396.580   36.240   
四分位距： 3.596   12.500   12.910    0.000    0.175    0.740   49.100    3.112   20.000   387.000    2.800   20.900   10.030   


## 数据预处理1
找出其异常数据，这里可认为数据集服从正态分布，因此剔除掉每个特征中：数据<(均值-3*标准差)或数据>(均值+3*标准差)，并将其替换为当前特征的均值

In [6]:
error = preprocessing.find_error_data(X)
error.find_error_var()
error.deal_error()
X = error.data
preprocessing.data_analysis(error.data)

均值： 2.784    9.145   11.137    0.002    0.555    6.284   68.575    3.724    9.549   408.237   18.456   373.350   12.424   
中位数： 0.257    0.000    9.690    0.000    0.538    6.212   77.500    3.207    5.000   330.000   19.050   391.440   11.360   
标准差： 5.048   19.114    6.860    0.010    0.116    0.636   28.149    1.980    8.707   168.537    2.165   51.738    6.757   
极差：28.649   80.000   27.280    0.067    0.486    4.007   97.100    8.093   23.000   524.000    9.400   313.450   32.290   
四分位距： 3.218   10.267   12.910    0.000    0.175    0.717   49.100    3.017   20.000   387.000    2.800   20.900    9.660   


## 数据预处理2
由于要使用梯度下降法，因此对数据进行归一化，即利用线性变换，把数据映射在（0，1）之间，便于梯度下降收敛更快。公式如下：**所有数据 - 最小值 / (最大值 - 最小值)**

In [7]:
# X = X[:, :6]
stand = preprocessing.Normalization()
X = stand.fit_transform(X)
X_train, X_test, y_train, y_test = divide_data.train_test_split(X, y, 0.2)

## 训练模型以及模型评估
分别使用最小二乘法和梯度下降法对模型进行训练以及评估，得到下面评估数据。从中可以看出最小二乘法效果优于梯度下降法，可能是梯度下降过程中在最低点处震荡导致。但是泛用性来说是梯度下降更好。

In [8]:
lr = linearregression.least_squares()
lr.fit(X_train, y_train)
y_pr = lr.predict(X_test)
ass = reg.reg_assessment(y_pr, y_test)
ass.assessment()

均方误差（MSE）： 27.612275391192917
平均绝对误差（MAE）： 3.594690396208019
均方根误差（RMSE）： 5.254738375142279
平均绝对百分比误差（MAPE）： 0.1907781482171757
R平方： 0.9950387324722576


In [9]:
gd = linearregression.gradient_descent()
gd.fit(X_train, y_train.reshape(-1,1))
y_pr = gd.predict(X_test)
ass = reg.reg_assessment(np.squeeze(y_pr), y_test)
ass.assessment()

均方误差（MSE）： 49.64566077686399
平均绝对误差（MAE）： 5.042302316374826
均方根误差（RMSE）： 7.045967696268837
平均绝对百分比误差（MAPE）： 0.2581019767263798
R平方： 0.9910798584609174


## 对比
对比sklearn中的线性回归，看看结果如何。

和最小二次的差不多吧

In [10]:
from sklearn.linear_model import LinearRegression
sk = LinearRegression()
sk.fit(X_train, y_train)
y_pr = sk.predict(X_test)
ass = reg.reg_assessment(y_pr, y_test)
ass.assessment()

均方误差（MSE）： 27.61227539119466
平均绝对误差（MAE）： 3.594690396208539
均方根误差（RMSE）： 5.254738375142445
平均绝对百分比误差（MAPE）： 0.1907781482172065
R平方： 0.9950387324722574


## 正则化-岭回归
![](过拟合.jpeg)
由于L1正则化在最小二乘和梯度下降法均无法实现，这里使用L2正则化——岭回归

岭回归，正则化技术的一种，用于处理过拟合问题。相对于线性回归增多了一个惩罚项$l_2$，其代价函数为
$$J(\theta)=\frac{1}{2m}[\sum_{i=1}^m(h_\theta(x^{(i)})-y^{(i)})^2+\lambda\sum_{j=1}^n\theta_j^2]$$
导数为
$$\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})\cdot x_i)+\frac{\lambda}{m}\theta_j$$
其中$\alpha$为控制模型复杂度的因子，可看做收缩率的大小。α越大，收缩率越大。

由于加了惩罚项，$\theta_j$的下降过程会比其他参数相对更快，直到其接近0则可以忽略其存在。
矩阵形式:
![图片](http://www.ai-start.com/ml2014/images/71d723ddb5863c943fcd4e6951114ee3.png)
**但是发现加了正则化项效果更差了，看来不是过拟合所致**

In [11]:
stand1 = linearregression.least_squares()
stand1.fit(X_train, y_train, 0.0001)
y_pr = stand1.predict(X_test)
ass = reg.reg_assessment(y_pr, y_test)
ass.assessment()

均方误差（MSE）： 28.038198516731747
平均绝对误差（MAE）： 3.7490885850630074
均方根误差（RMSE）： 5.2951108125073025
平均绝对百分比误差（MAPE）： 0.20190553799438446
R平方： 0.9949622042418922


In [12]:
stand2 = linearregression.gradient_descent()
stand2.fit(X_train, y_train.reshape(-1,1), lam=0.0001)
y_pr = stand2.predict(X_test)
ass = reg.reg_assessment(np.squeeze(y_pr), y_test)
ass.assessment()

均方误差（MSE）： 47.28989607739201
平均绝对误差（MAE）： 4.855375671832501
均方根误差（RMSE）： 6.87676494271776
平均绝对百分比误差（MAPE）： 0.25582244224784767
R平方： 0.991503133208866
